# Healthcare Cost Prediction - Exploratory Data Analysis

This notebook performs comprehensive exploratory data analysis on healthcare cost data.

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print('Libraries imported successfully!')

In [ ]:
# Load data
df = pd.read_csv('../data/raw/patient_data.csv')

print(f'Dataset shape: {df.shape}')
print(f'\nColumns: {df.columns.tolist()}')
df.head()

## 1. Data Quality Assessment

In [ ]:
# Check data types and missing values
print('Data Types:\n')
print(df.dtypes)
print('\n' + '='*50)
print('\nMissing Values:\n')
print(df.isnull().sum())
print('\n' + '='*50)
print('\nBasic Statistics:\n')
df.describe()

## 2. Cost Distribution Analysis

In [ ]:
# Cost distribution
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Histogram
axes[0, 0].hist(df['total_annual_cost'], bins=50, edgecolor='black', alpha=0.7)
axes[0, 0].set_title('Cost Distribution', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Annual Cost ($)')
axes[0, 0].set_ylabel('Frequency')

# Log-scale histogram
axes[0, 1].hist(np.log10(df['total_annual_cost']+1), bins=50, edgecolor='black', alpha=0.7, color='green')
axes[0, 1].set_title('Cost Distribution (Log Scale)', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Log10(Annual Cost)')
axes[0, 1].set_ylabel('Frequency')

# Box plot
axes[1, 0].boxplot(df['total_annual_cost'])
axes[1, 0].set_title('Cost Box Plot', fontsize=14, fontweight='bold')
axes[1, 0].set_ylabel('Annual Cost ($)')

# Q-Q plot
stats.probplot(df['total_annual_cost'], dist="norm", plot=axes[1, 1])
axes[1, 1].set_title('Q-Q Plot', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

# Summary statistics
print(f'Mean: ${df["total_annual_cost"].mean():,.2f}')
print(f'Median: ${df["total_annual_cost"].median():,.2f}')
print(f'Std Dev: ${df["total_annual_cost"].std():,.2f}')
print(f'Min: ${df["total_annual_cost"].min():,.2f}')
print(f'Max: ${df["total_annual_cost"].max():,.2f}')

## 3. Risk Category Analysis

In [ ]:
# Risk category distribution
risk_counts = df['risk_category'].value_counts()

fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Bar chart
risk_counts.plot(kind='bar', ax=axes[0], color=['green', 'yellow', 'orange', 'red'])
axes[0].set_title('Risk Category Distribution', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Risk Category')
axes[0].set_ylabel('Number of Patients')
axes[0].tick_params(axis='x', rotation=45)

# Pie chart
axes[1].pie(risk_counts, labels=risk_counts.index, autopct='%1.1f%%', 
           colors=['green', 'yellow', 'orange', 'red'], startangle=90)
axes[1].set_title('Risk Category Proportions', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

# Cost by risk category
print('\nAverage Cost by Risk Category:\n')
print(df.groupby('risk_category')['total_annual_cost'].agg(['count', 'mean', 'median', 'std']))

## 4. Demographic Analysis

In [ ]:
# Age and cost relationship
fig = px.scatter(df, x='age', y='total_annual_cost', color='risk_category',
                size='bmi', hover_data=['chronic_conditions_count'],
                title='Age vs Cost by Risk Category',
                labels={'total_annual_cost': 'Annual Cost ($)'})
fig.show()

# BMI distribution by gender
fig, ax = plt.subplots(figsize=(12, 6))
df.boxplot(column='bmi', by='gender', ax=ax)
plt.title('BMI Distribution by Gender', fontsize=14, fontweight='bold')
plt.suptitle('')
plt.show()

# Smoker impact
smoker_costs = df.groupby('smoker')['total_annual_cost'].mean()
print(f'\nAverage cost - Non-smokers: ${smoker_costs[0]:,.2f}')
print(f'Average cost - Smokers: ${smoker_costs[1]:,.2f}')
print(f'Smoking premium: ${smoker_costs[1] - smoker_costs[0]:,.2f} ({(smoker_costs[1]/smoker_costs[0]-1)*100:.1f}%)')

## 5. Clinical Factors Analysis

In [ ]:
# Chronic conditions impact
chronic_costs = df.groupby('chronic_conditions_count')['total_annual_cost'].mean().reset_index()

fig = px.bar(chronic_costs, x='chronic_conditions_count', y='total_annual_cost',
            title='Average Cost by Number of Chronic Conditions',
            labels={'chronic_conditions_count': 'Number of Chronic Conditions',
                   'total_annual_cost': 'Average Annual Cost ($)'})
fig.show()

# Healthcare utilization
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Office visits
axes[0].scatter(df['previous_office_visits'], df['total_annual_cost'], alpha=0.5)
axes[0].set_title('Office Visits vs Cost')
axes[0].set_xlabel('Number of Office Visits')
axes[0].set_ylabel('Annual Cost ($)')

# ER visits
axes[1].scatter(df['previous_er_visits'], df['total_annual_cost'], alpha=0.5, color='orange')
axes[1].set_title('ER Visits vs Cost')
axes[1].set_xlabel('Number of ER Visits')
axes[1].set_ylabel('Annual Cost ($)')

# Hospitalizations
axes[2].scatter(df['previous_hospitalizations'], df['total_annual_cost'], alpha=0.5, color='red')
axes[2].set_title('Hospitalizations vs Cost')
axes[2].set_xlabel('Number of Hospitalizations')
axes[2].set_ylabel('Annual Cost ($)')

plt.tight_layout()
plt.show()

## 6. Correlation Analysis

In [ ]:
# Select numeric columns
numeric_cols = df.select_dtypes(include=[np.number]).columns
correlation_matrix = df[numeric_cols].corr()

# Heatmap
plt.figure(figsize=(14, 10))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
           center=0, square=True, linewidths=1)
plt.title('Correlation Matrix', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

# Top correlations with cost
cost_correlations = correlation_matrix['total_annual_cost'].sort_values(ascending=False)
print('\nTop 10 Correlations with Annual Cost:\n')
print(cost_correlations[1:11])

## 7. Insurance Analysis

In [ ]:
# Cost by insurance type
insurance_analysis = df.groupby('insurance_type').agg({
   'total_annual_cost': ['count', 'mean', 'median'],
   'patient_responsibility': 'mean',
   'insurance_paid': 'mean'
}).round(2)

print('Insurance Type Analysis:\n')
print(insurance_analysis)

# Visualize
fig = px.box(df, x='insurance_type', y='total_annual_cost',
            title='Cost Distribution by Insurance Type',
            labels={'insurance_type': 'Insurance Type',
                   'total_annual_cost': 'Annual Cost ($)'})
fig.show()

## 8. Statistical Tests

In [ ]:
# T-test: Smokers vs Non-smokers
smokers = df[df['smoker'] == 1]['total_annual_cost']
non_smokers = df[df['smoker'] == 0]['total_annual_cost']
t_stat, p_value = stats.ttest_ind(smokers, non_smokers)

print(f'T-test: Smokers vs Non-smokers')
print(f'T-statistic: {t_stat:.4f}')
print(f'P-value: {p_value:.4f}')
print(f'Significant difference: {"Yes" if p_value < 0.05 else "No"}\n')

# ANOVA: Cost across insurance types
insurance_groups = [df[df['insurance_type'] == ins]['total_annual_cost'] 
                   for ins in df['insurance_type'].unique()]
f_stat, p_value = stats.f_oneway(*insurance_groups)

print(f'\nANOVA: Cost across insurance types')
print(f'F-statistic: {f_stat:.4f}')
print(f'P-value: {p_value:.4f}')
print(f'Significant difference: {"Yes" if p_value < 0.05 else "No"}')

## 9. Key Insights Summary

In [ ]:
print('='*60)
print('KEY INSIGHTS')
print('='*60)

high_risk_pct = (df['risk_category'].isin(['High', 'Catastrophic']).sum() / len(df)) * 100
high_risk_cost_pct = (df[df['risk_category'].isin(['High', 'Catastrophic'])]['total_annual_cost'].sum() / 
                     df['total_annual_cost'].sum()) * 100

print(f'\n1. {high_risk_pct:.1f}% of patients are high-risk but account for {high_risk_cost_pct:.1f}% of total costs')
print(f'\n2. Average cost increases by ${df.groupby("chronic_conditions_count")["total_annual_cost"].mean().diff().mean():,.0f} per chronic condition')
print(f'\n3. Smokers cost ${smoker_costs[1] - smoker_costs[0]:,.0f} more annually than non-smokers')
print(f'\n4. Top 3 cost drivers: {cost_correlations[1:4].index.tolist()}')
print(f'\n5. Cost variation (CV): {(df["total_annual_cost"].std() / df["total_annual_cost"].mean()):.2f}')
print('\n' + '='*60)